In [1]:
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import PromptTemplate
from langchain_community.vectorstores.faiss import FAISS
from langchain.memory import ConversationBufferMemory
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain_openai.chat_models import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
import os

In [2]:
import os

def carrega_vector_store():
    embedding_model = OpenAIEmbeddings()
    
    # Carregar o FAISS index salvo no diretório
    vector_store = FAISS.load_local("Algorithmic_Trading", embedding_model, allow_dangerous_deserialization=True)
    
    return vector_store

In [13]:
"""def carrega_vector_store(pasta):
    embedding_model = OpenAIEmbeddings()
    vector_stores = []
    
    # Listar todos os arquivos na pasta
    for arquivo in os.listdir(pasta):
        caminho_arquivo = os.path.join(pasta, arquivo)
        if caminho_arquivo:
            try:

                # Carregar o FAISS index salvo no diretório
                caminho_arquivo = f"vector/{arquivo}"
                vector_store = FAISS.load_local(caminho_arquivo, embedding_model, allow_dangerous_deserialization=True)
                #vector_stores.append(vector_store)
            except Exception as e:
                print(f"Não foi possível carregar o arquivo {arquivo}: {e}")
    
    # Combinar todos os vector_stores em um só
    if vector_store:
        # Exemplo de combinação simples: concatenar todos os índices
        #combined_vector_store = FAISS.concatenate(vector_stores)
        return vector_store
    else:
        vector_stores.__annotations__
        print(vector_stores.__annotations__)"""

In [3]:
# Carregar o vector_store salvo

# Função para criar o chatbot com conversação e retrieval
def cria_chatbot():
    # Carregar o vector_store
    vector_store = carrega_vector_store("vector/")
    
    # Criar o retriever a partir do vector_store
    retriever = vector_store.as_retriever(
        search_type="similarity",  # ou configure conforme sua preferência
        search_kwargs={"k": 5}  # top k resultados
    )

    # Criar modelo de linguagem (LLM) OpenAI
    chat = ChatOpenAI(model="gpt-3.5-turbo")

    # Criar memória de conversação
    memory = ConversationBufferMemory(
        return_messages=True,
        memory_key='chat_history',
        output_key='answer'
    )

    # Prompt de exemplo, pode ser ajustado conforme sua necessidade
    prompt_template = """
    You are an AI assistant designed to help students with their questions. Use the context provided below to answer the user's questions accurately. If you do not know the answer, simply state that you do not know and refrain from making up information. Whenever possible, include references to the relevant context used to formulate your response.
    Context: {context}
    ----
    Question: {question}
    ----
    Answer in a conversational and informative way.
    """
    prompt = PromptTemplate.from_template(prompt_template)

    # Criar a chain de conversação com retrieval
    chat_chain = ConversationalRetrievalChain.from_llm(
        llm=chat,
        memory=memory,
        retriever=retriever,
        return_source_documents=True,
        verbose=True,
        combine_docs_chain_kwargs={'prompt': prompt}
    )

    return chat_chain



In [4]:
import dotenv
from dotenv import load_dotenv, find_dotenv


_ = load_dotenv(find_dotenv())


In [5]:
# Função para interagir com o chatbot
def conversar_com_chatbot(chat_chain, pergunta):
    # Usar a chave correta 'question'
    resposta = chat_chain.invoke({
        "question": pergunta
    })
    return resposta['answer']

# Exemplo de uso
chatbot = cria_chatbot()
pergunta = "Tenho uma dúvida, voce poderia me ajudar a lembrar o que é blacktesting"
resposta = conversar_com_chatbot(chatbot, pergunta)
print(resposta)




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

    You are an AI assistant designed to help students with their questions. Use the context provided below to answer the user's questions accurately. If you do not know the answer, simply state that you do not know and refrain from making up information. Whenever possible, include references to the relevant context used to formulate your response.
    Context: . Regime shifts can spoil everything, and a few \nimportant historical examples will be highlighted.\nThe choice of a software platform for backtesting is also an important \nconsideration and needs to be tackled early on. A good choice not only will \nvastly increase your productivity, it will also allow you to backtest the broad-\nest possible spectrum of strategies in the broadest variety of asset classes. \n', '\n--- Página 20 ---\n2\nALGORITHMIC TRADING\nAnd it will reduce or eliminate the chances of committing the afor